In [1]:
import numpy as np
import pandas as pd
import load_rating_data as ld
from utils import RMSE 
import copy
from timeit import default_timer as timer
from sklearn.model_selection import train_test_split

In [2]:
df = ld.load_rating_data() #load_rating_1m()
ratio = 0.8
train, test = train_test_split(df, test_size=1-ratio)
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [3]:
X_train, Y_train =ld.one_hot_data(train, df)
X_test, Y_test = ld.one_hot_data(test, df)

In [4]:
import torch
from torch import optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [5]:
class MovielensDataset(Dataset):
    """
    torch.utils.data.Dataset 상속
    """
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return self.X.size(0)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [6]:
train_dataset = MovielensDataset(X = torch.FloatTensor(X_train),
                                y = torch.FloatTensor(Y_train))

In [7]:
class FactorizationMachine(nn.Module):
    
    def __init__(self, field_dims, latent_dims):
        super(FactorizationMachine, self).__init__()
        
        self.w_0 = nn.Parameter(nn.init.normal_(torch.zeros((1, ))), requires_grad=True) #(1, )
        self.w_i = nn.Parameter(nn.init.normal_(torch.zeros((1, field_dims)), std=1.0/field_dims), requires_grad = True) # (1, 4308)
        self.V = nn.Parameter(nn.init.normal_(torch.zeros((field_dims, latent_dims)), std=1.0/latent_dims), requires_grad = True) # (4308, 40)
        
        
    def forward(self, x):
        """
        batch shape (64, 4308)
        """
        temp_1 = self.w_0 + torch.matmul(x, self.w_i.T) # (64, 1)
        
        square_of_sum = torch.sum(torch.matmul(x, self.V), dim = 1) ** 2
        sum_of_square = torch.sum(torch.matmul(x, self.V) ** 2, dim = 1)
        temp_2 = (square_of_sum - sum_of_square).view(-1, 1)
        
        result = temp_1 + 0.5 * temp_2
        
        return result

In [8]:
model = FactorizationMachine(field_dims = X_train.shape[1], latent_dims = 20)
optimizer = optim.SGD(model.parameters(), lr=0.001)
loss_function = nn.MSELoss()
batch_size = 64
n_epochs = 100

In [10]:
for epoch_id in range(n_epochs):
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    model.train()
    total_loss = 0
    for batch_idx, batch in enumerate(train_loader):

        X, y = batch[0], batch[1]

        optimizer.zero_grad()
        y_pred = model(X)
        loss = loss_function(y_pred.view(-1), y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss
        
    model.eval()
    
    y_test = model(torch.FloatTensor(X_test))
    y_test = y_test.cpu().detach().numpy()
    rmse = np.sqrt(np.mean((y_test - Y_test)**2))
        
    if ((epoch_id + 1) % 10 == 0 ):
        print ('Epoch {} of {}, training Loss: {:.4f}, RMSE: {:.4f}'.format(epoch_id + 1, n_epochs, total_loss, rmse))

Epoch 1 of 100, training Loss: 1597.0929, RMSE: 1.1184
Epoch 2 of 100, training Loss: 1562.6223, RMSE: 1.1144
Epoch 3 of 100, training Loss: 1552.3138, RMSE: 1.1111
Epoch 4 of 100, training Loss: 1542.5454, RMSE: 1.1079
Epoch 5 of 100, training Loss: 1533.1188, RMSE: 1.1047
Epoch 6 of 100, training Loss: 1523.9510, RMSE: 1.1017
Epoch 7 of 100, training Loss: 1515.0883, RMSE: 1.0987
Epoch 8 of 100, training Loss: 1506.5736, RMSE: 1.0959
Epoch 9 of 100, training Loss: 1498.3954, RMSE: 1.0932
Epoch 10 of 100, training Loss: 1490.5292, RMSE: 1.0906


KeyboardInterrupt: 